<a href="https://colab.research.google.com/github/Gauravpareek11/Stock_prediction/blob/main/StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf


In [ ]:
data=yf.download("TATAMOTORS.NS",start="2018-01-01",interval='1d')
# stock="SBIN.NS"

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.shape

(985, 6)

In [ ]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,430.950012,436.399994,422.250000,424.450012,424.450012,6807536
2018-01-02,428.850006,440.850006,422.000000,439.299988,439.299988,15331261
2018-01-03,440.399994,441.399994,431.950012,433.899994,433.899994,9794953


In [ ]:
# import requests
# url="https://www.alphavantage.co/query?function=RSI&symbol={}&interval=daily&time_period=5&series_type=close&apikey=VBU6C9CV9HW3XJ7T"
# response=requests.get(url.format(stock))
# response=response.json()

In [ ]:
# response.keys()

In [ ]:
# rsi_data=pd.DataFrame.from_dict(response['Technical Analysis: RSI'],orient='index')
# rsi_data.head()
# rsi_data=rsi_data[rsi_data.index>='2018-01-02']

In [ ]:
# rsi_data=rsi_data['RSI'].astype(np.float64)

In [ ]:
# data=data.merge(rsi_data,left_index=True,right_index=True,how='inner')
# data.head()

In [ ]:
data.sort_index(inplace=True)

In [ ]:
data=data.loc[~data.index.duplicated(keep='first')]   #remove duplicate index

In [ ]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-22,458.149994,472.149994,458.149994,470.500000,470.500000,22730750
2021-12-23,476.000000,478.000000,470.950012,472.350006,472.350006,12116814
2021-12-24,474.899994,474.899994,460.000000,467.600006,467.600006,15487368
2021-12-27,465.700012,472.450012,460.100006,471.149994,471.149994,12557565
2021-12-28,475.200012,482.799988,472.649994,480.200012,480.200012,22856919


In [ ]:
data.isnull().sum() #checking for missing value

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,985.000000,985.000000,985.000000,985.000000,985.000000,9.850000e+02
mean,231.226345,235.253959,226.756447,230.660051,230.660051,4.191146e+07
std,107.105319,108.433672,105.500781,106.763436,106.763436,3.965734e+07
min,66.500000,66.900002,63.500000,65.300003,65.300003,3.820313e+06
25%,152.949997,156.250000,150.000000,151.800003,151.800003,1.533126e+07
50%,185.949997,188.449997,182.800003,185.350006,185.350006,3.036126e+07
75%,308.899994,313.649994,304.049988,308.399994,308.399994,5.588726e+07
max,531.450012,536.700012,520.250000,530.150024,530.150024,3.905778e+08


In [ ]:
import plotly.graph_objects as go
# check the trend in closing value
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.index,y=data['Close'],mode='lines'))
fig.update_layout(height=500,width=900,xaxis_title='Date',yaxis_title='close')
fig.show()

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.index,y=data['Volume'],mode='lines'))
fig.update_layout(height=500,width=900,xaxis_title='Date',yaxis_title='Volume')
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [ ]:
data=data[['Close','Volume']]
data.head()

,Close,Volume
Date,,
2018-01-01,424.450012,6807536
2018-01-02,439.299988,15331261
2018-01-03,433.899994,9794953
2018-01-04,429.950012,8395377
2018-01-05,431.600006,7021611


In [ ]:
test_length=data[(data.index>="2021-01-01")].shape[0]

In [ ]:
def CreateFeautres_and_Targets(data,feautre_length):
  x=[]
  y=[]
  for i in tnrange(len(data)-feautre_length):
    x.append(data.iloc[i:i+feautre_length:].values)
    y.append(data['Close'].values[i+feautre_length])

  x=np.array(x)
  y=np.array(y)
  return x,y

In [ ]:
x,y=CreateFeautres_and_Targets(data,32)

  0%|          | 0/953 [00:00<?, ?it/s]

In [ ]:
x.shape,y.shape

((953, 32, 2), (953,))

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=x[:-test_length],x[-test_length:],y[:-test_length],y[-test_length:]

In [ ]:
Xtrain.shape,Ytrain.shape

((708, 32, 2), (708,))

In [ ]:
Xtest.shape,Ytest.shape

((245, 32, 2), (245,))

In [ ]:
#scaling vectors to sacle a vector with multiple dimensions
class MultiDimensionScaler():
  def __init__(self):
    self.scalers=[]
  def fit_transform(self,X):
    total_dims=X.shape[2]
    for i in range(total_dims):
      Scaler=MinMaxScaler()
      X[:,:,i]=Scaler.fit_transform(X[:,:,i])
      self.scalers.append(Scaler)
    return X
  def transform(self,X):
    for i in range(X.shape[2]):
      X[:,:,i]=self.scalers[i].transform(X[:,:,i])
    return X

In [ ]:
Feautre_Scaler=MultiDimensionScaler()
Xtrain=Feautre_Scaler.fit_transform(Xtrain)
Xtest=Feautre_Scaler.fit_transform(Xtest)

In [ ]:
Target_Scaler=MinMaxScaler()
Ytrain=Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest=Target_Scaler.fit_transform(Ytest.reshape(-1,1))

In [ ]:
def save_object(obj,name:str):
  pickle_out=open(f"{name}.pck","wb")
  pickle.dump(obj,pickle_out)
  pickle_out.close()

def load_object(name:str):
  pickle_in=open(f"{name}.pck","rb")
  data=pickle.load(pickle_in)
  return data

In [ ]:

save_object(Feautre_Scaler,"Feautre_scaler")
save_object(Target_Scaler,"Target_scaler")

In [ ]:
# from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
# save_best=ModelCheckpoint("best_weights.h5",monitor='val_loss',save_best_only=True,save_weights_only=True)
# reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.25,patience=5,min_lr=0.00001,verbose=1)

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense,Dropout,LSTM,Bidirectional
# model=Sequential()
# model.add(Bidirectional(LSTM(512,return_sequences=True,recurrent_dropout=0.1,
#                              input_shape=(32,2))))
# model.add(LSTM(256,recurrent_dropout=0.1))
# model.add(Dropout(0.3))
# model.add(Dense(64,activation='elu'))
# model.add(Dropout(0.3))
# model.add(Dense(32,activation='elu'))
# model.add(Dense(1,activation='linear'))

In [ ]:
# optimizer=tf.keras.optimizers.SGD(learning_rate=0.002)
# model.compile(loss='mse',optimizer=optimizer)


In [ ]:
# history=model.fit(Xtrain,Ytrain,
#                   epochs=10,
#                   batch_size=1,
#                   verbose=1,
#                   shuffle=False,
#                   validation_data=(Xtest,Ytest),
#                   callbacks=[reduce_lr,save_best])

In [ ]:
# model.load_weights("best_weights.h5")

In [ ]:
from tensorflow.python.keras.saving.save import load_model
model=load_model('/content/drive/MyDrive/Colab Notebooks/stock.h5')

In [ ]:
predictions=model.predict(Xtest)

In [ ]:
predictions=Target_Scaler.inverse_transform(predictions)
Actual=Target_Scaler.inverse_transform(Ytest)
Actual.shape

(245, 1)

In [ ]:
predictions=np.squeeze(predictions,axis=1)
Actual=np.squeeze(Actual,axis=1)
print(predictions[0])
print(Actual[0])

242.14305
186.5


In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.index[-test_length:],y=predictions,mode='lines',name='predicted'))
fig.add_trace(go.Scatter(x=data.index[-test_length:],y=Actual,mode='lines',name='Actual'))
fig.update_layout(height=500,width=900,xaxis_title='Date',yaxis_title='Volume')
fig.show()

In [ ]:
# model.save('/content/drive/MyDrive/Colab Notebooks/stock.h5')

In [ ]:
def predictStock(model,DataFrame,previousDate,feautre_length=32):
  idx_location=DataFrame.index.get_loc(previousDate)
  Feautres=DataFrame.iloc[idx_location-feautre_length:idx_location,:].values
  Feautres=np.expand_dims(Feautres,axis=0)
  Feautres=Feautre_Scaler.transform(Feautres)
  predictions=model.predict(Feautres)
  predictions=Target_Scaler.inverse_transform(predictions)
  return predictions[0][0]

In [ ]:
predictStock(model,data,'2021-12-27')

552.7696